In [ ]:
import numpy as np
import nibabel as nib
import pandas as pd
from keras.models import load_model

def squeeze_dim(img):
    if img.ndim == 4: 
        if img.shape[0] == 1:
            img = np.squeeze(img,axis=0)
        if img.shape[1] == 1:
            img = np.squeeze(img,axis=1)
        if img.shape[2] == 1:
            img = np.squeeze(img,axis=2)
        if img.shape[3] == 1:
            img = np.squeeze(img,axis=3)
    else:
        pass
    return img
def crop(img, a , b, c):
    '''
    有一个缺点就是 a,b,c的值只能是偶数,即便是设置为奇数,返回的shape也是(a-1,b-1,c-1)之类的偶数形状
    '''
    img_shape = img.shape
    img_a = img_shape[0]
    img_b = img_shape[1]
    img_c = img_shape[2]
    img_res = img[img_a/2-a/2:img_a/2+a/2, img_b/2-b/2:img_b/2+b/2, img_c/2-c/2:img_c/2+c/2]
    return img_res
def decrop(in_img,shell):
    """
    shell 是一个全零的壳子矩阵,用来装预测的图形
    in_img 是预测得到的被剪切图像
    """
    in_img_shape = in_img.shape
    shell_shape = shell.shape
    a = in_img_shape[0]
    b = in_img_shape[1]
    c = in_img_shape[2]
    img_a = shell_shape[0]
    img_b = shell_shape[1]
    img_c = shell_shape[2]
    shell[img_a/2-a/2:img_a/2+a/2, img_b/2-b/2:img_b/2+b/2, img_c/2-c/2:img_c/2+c/2] = in_img
    return shell
def make_csv(floder= 'sample/image',save_name='./test.csv'):
    """
    get all filename in floder to make a csv_file
    input:
        floder is a path(type = string)
    """
    list_file = os.listdir(floder)
    csvfile = file(save_name,'wb')
    writer = csv.writer(csvfile)
    writer.writerow(['path'])
    list_file.sort()
    for file_name in list_file:
        writer.writerow([file_name])
    csvfile.close()
def loadDataGeneral(df,target_floder, append_coord):
    """
    这个是将待测图片整体读入,然后记录shape 其中做了整体的归一化
    This function loads data stored in nifti format. Data should already be of
    appropriate shape.

    Inputs:
    - df: Pandas dataframe with two columns: image filenames and ground truth filenames.
    - path: Path to folder containing filenames from df.
    - append_coords: Whether to append coordinate channels or not.
    Returns:
    - X: Array of 3D images with 1 or 4 channels depending on `append_coords`.
    - y: Array of 3D masks with 1 channel.
    """
   
    X, shape_list = [], []
    for i, item in df.iterrows():

        img_path = ''.join([str(target_floder), item[0]])

        nii_img = nib.load(img_path)
        img = nii_img.get_data()
        shape = img.shape
        img = squeeze_dim(img)
        img = crop(img,96,96,80)

        img = np.array(img, dtype=np.float64)
        img -= img.mean()
        img /= img.std()
        X.append(img)
        shape_list.append(shape)

    X = np.array(X)
    X -= X.mean()    #如果不做整体的归一化,则注释这两句
    X /= X.std()
    X = np.expand_dims(X, -1)  # X.shape  --> (4,128,128,64,1)
    
    # Option to append coordinates as additional channels
    if append_coord:
        n = X.shape[0]
        inpShape = X.shape[1:]
        xx = np.empty(inpShape)
        for i in xrange(inpShape[1]):
            xx[:, i, :, 0] = i
        yy = np.empty(inpShape)
        for i in xrange(inpShape[0]):
            yy[i, :, :, 0] = i
        zz = np.empty(inpShape)
        for i in xrange(inpShape[2]):
            zz[:, :, i, 0] = i
        X = np.concatenate([X, np.array([xx] * n), np.array([yy] * n), np.array([zz] * n)], -1)

    print '### Dataset loaded'
    print '\t{}'.format(target_floder)
    print '\t{}\t'.format(X.shape)
    print '\tX:{:.1f}-{:.1f}\t'.format(X.min(), X.max())
    return X, shape_list
def prediction(csv_path, target_floder,append_coords = False):
    
    """
    载入的模型名称是my_model.hdf5
    用以针对一个文件夹下边的文件进行整体的预测,并保存预测图像到与target_floder文件夹相同子目录pred文件夹下
    这个做的对原图(未经过裁剪的图像)进行的推断,得到的是与原图相匹配的pred文件
    """
    df = pd.read_csv(csv_path) 
    X,shape_list= loadDataGeneral(df,target_floder, append_coords)
    n_test = X.shape[0]   
    model_name = 'my_model.hdf5' # Model should be trained with the same `append_coords`
    model = load_model(model_name)
    pred = model.predict(X, batch_size=1)[..., 1]  
    save_pred_floder = target_floder[:target_floder.rfind("/")]+'/pred/'
    if not os.path.exists(save_pred_floder):
        os.makedirs(save_pred_floder)
    for i in range(n_test):

        pr = pred[i] > 0.5 # binary prediction
        shell = np.zeros(shape_list[i])
        pr = decrop(pr,shell)

        tImg = nib.load(target_floder + df.ix[i].path)
        nib.save(nib.Nifti1Image(255 * pr.astype('float'), affine=tImg.get_affine()),
                 save_pred_floder + df.ix[i].path[:-7]+'-pred.nii.gz')

if __name__ == "__main__":
    csv_path = 'new_rotate/test.csv'
    target_floder = '/home/yanyu/competition/new_rotate/test_data'
    prediction(csv_path,target_floder)
    

In [15]:
#这个文件是用来对生成的pred文件和gt两个文件夹中的图像进行求dice值和iou
#
import numpy as np
import nibabel as nib
import pandas as pd
import csv
"""
比较上个文件生成的pred和gt,对比dice和iou
"""
def make_csv(floder= 'sample/image',save_name='test.csv'):
    """
    get all filename in floder to make a csv_file
    input:
        floder is a path(type = string)
    """
    list_file = os.listdir(floder)
    csvfile = file(save_name,'wb')
    writer = csv.writer(csvfile)
    writer.writerow(['path'])
    list_file.sort()
    for file_name in list_file:
        writer.writerow([file_name])
    csvfile.close()

def IoU(y_true, y_pred):
    assert y_true.dtype == bool and y_pred.dtype == bool
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.logical_and(y_true_f, y_pred_f).sum()
    union = np.logical_or(y_true_f, y_pred_f).sum()
    return (intersection + 1) * 1. / (union + 1)

def Dice(y_true, y_pred):
    assert y_true.dtype == bool and y_pred.dtype == bool
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.logical_and(y_true_f, y_pred_f).sum()
    return (2. * intersection + 1.) / (y_true.sum() + y_pred.sum() + 1.)

if __name__ == "__main__":
    gt_floder = "path/to/ground/truth/"
    pr_floder = "path/to/prediction/file/"  #注意最后的/
    make_csv(gt_floder,"gt.csv")
    make_csv(pr_floder,'pr.csv')
    df1 = pd.read_csv("gt.csv")
    df2 = pd.read_csv('pr.csv')
    IOU = []
    DICE = []
    for [i,item1],[j,item2] in zip(df1.iterrows(),df2.iterrows()):
        gt = nib.load(gt_floder+item1[0]).get_data().astype(np.bool)
        pr = nib.load(pr_floder+item2[0]).get_data().astype(np.bool)
        iou = IoU(gt,pr)
        dice = Dice(gt,pr)
        print("The number of {}".format(i))
        print("iou:{:.5f}\t dice:{:.5f}".format(iou,dice))
        IOU.append(iou)
        DICE.append(dice)
    print("mean iou is:")
    print(np.array(IOU).mean())
    print("mean dice is:")
    print(np.array(DICE).mean())
    
        

In [21]:
make_csv('../Demo/crop/image','../Demo/crop/test1.csv')
make_csv('../Demo/crop/temp','../Demo/crop/test2.csv')
print("done")
df1 = pd.read_csv('../Demo/crop/test1.csv')
df2 = pd.read_csv('../Demo/crop/test2.csv')
for [i,item1],[j,item2] in zip(df1.iterrows(),df2.iterrows()):
    print("i:{}\t item1:{}\t item2:{}".format(i,item1[0],item2[0]))

done
i:0	 item1:ADNI_002_S_1018_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070217030439623_S23128_I40817resizecrop.nii.gz	 item2:t1.nii.gz
i:1	 item1:ADNI_003_S_0907_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070501171753634_S19728_I52781resizecrop.nii.gz	 item2:t2.nii.gz
i:2	 item1:ADNI_003_S_0981_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070501171453195_S20753_I52776resizecrop.nii.gz	 item2:t3.nii.gz
i:3	 item1:ADNI_005_S_0448_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20061212165540546_S14032_I32875resizecrop.nii.gz	 item2:t4.nii.gz


In [24]:
a=[1,2,3,4]
print(a)
b = np.array(a)
print(b)

[1, 2, 3, 4]
[1 2 3 4]
